In [3]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocess as mp
import glob
import time
from tqdm import tqdm
import os
import sys
import pandas as pd
import geopandas as gpd
import warnings
import matplotlib.pyplot as plt
import csv
import pyproj
from datetime import datetime

data_path = '/mnt/inca/ai4sh_data.harmo'

## read in, and add temporal, depth information

### year 2009, 2012, and 2015

In [4]:
# lucas 2009
lucas2009 = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS_2009/LUCAS_TOPSOIL_v1_2009.csv', low_memory=False)
lucas2009 = lucas2009.rename(columns={'POINT_ID':'point_id','pH_in_H2O':'ph_h2o','pH_in_CaCl2':'ph_cacl2',
                                      'OC':'oc','CaCO3':'caco3','GPS_LAT':'latg','GPS_LONG':'long'})
lucas2009['time']=2009
lucas2009['hzn_top'] = 0
lucas2009['hzn_btm'] = 20
lucas2009.to_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.2009_harmonized.csv',index=False)

In [5]:
# lucas 2012
bulgaria = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS_Romania_Bulgaria_2012/Bulgaria.csv')
bulgaria = bulgaria.rename(columns={'POINT_ID':'point_id','pHinH2O':'ph_h2o','pHinCaCl2':'ph_cacl2',
                                      'OC':'oc','CaCO3':'caco3','GPS_Y_LAT':'latg','GPS_X_LONG':'long'})

romania = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS_Romania_Bulgaria_2012/Romania.csv')
romania = romania.rename(columns={'POINTID':'point_id','pHinH2O':'ph_h2o','pHinCaCl2':'ph_cacl2',
                                      'OC':'oc','CaCO3':'caco3','GPSYLAT':'latg','GPSXLONG':'long'})

lucas2012 = pd.concat([bulgaria, romania])
lucas2012['hzn_top'] = 0
lucas2012['hzn_btm'] = 20
lucas2012['time'] = 2012
lucas2012.to_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.2012_harmonized.csv',index=False)

In [6]:
# lucas 2015
lucas2015 = gpd.read_file(f'{data_path}/LUCAS_TOPSOIL/LUCAS2015_topsoildata_20200323/LUCAS_Topsoil_2015_20200323-shapefile/LUCAS_Topsoil_2015_20200323.shp')
lucas2015 = lucas2015.rename(columns={'Point_ID':'point_id','pH_H20':'ph_h2o','pH_CaCl2':'ph_cacl2','OC':'oc','CaCO3':'caco3',
                                     'Clay':'clay','Sand':'sand','Silt':'silt'})
lucas2015['time'] = 2015
lucas2015['latg'] = lucas2015['geometry'].y
lucas2015['long'] = lucas2015['geometry'].x
lucas2015['hzn_top'] = 0
lucas2015['hzn_btm'] = 20
lucas2015.to_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.2015_harmonized.csv',index=False)


### year 2018 is a bit more complicated, since it not only sample points from top soil, but also 20-30 cm

In [18]:
# get the lucas 2018, and the rows with 20-30cm depth
lucas2018 = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS-SOIL-2018-data-report-readme-v2/LUCAS-SOIL-2018-v2/LUCAS-SOIL-2018.csv', low_memory=False)
lucas2018 = lucas2018.rename(columns={'POINTID':'point_id','pH_H2O':'ph_h2o','pH_CaCl2':'ph_cacl2',
                                      'OC':'oc','CaCO3':'caco3','TH_LAT':'latg','TH_LONG':'long'})

dep2018 = lucas2018[['point_id','latg','long','OC (20-30 cm)','CaCO3 (20-30 cm)']]
dep2018 = dep2018.rename(columns={'OC (20-30 cm)':'oc','CaCO3 (20-30 cm)':'caco3'})
dep2018 = dep2018.dropna(subset=['oc', 'caco3'], how='all')
dep2018['Depth'] = '20-30 cm'
lucas2018['point_id'] = lucas2018['point_id'].astype(int)
lucas2018 = pd.concat([lucas2018,dep2018],axis=0).reset_index(drop=True)
lucas2018 = lucas2018.drop(columns = ['OC (20-30 cm)','CaCO3 (20-30 cm)'])

# get all the bulk density rows and combine them with
bd2018 = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS-SOIL-2018-data-report-readme-v2/LUCAS-SOIL-2018-v2/BulkDensity_2018_final-2.csv', low_memory=False)
bd =  pd.DataFrame(columns=['point_id','Depth','bulk_density'])
for col in ['BD 0-10','BD 10-20', 'BD 20-30', 'BD 0-20']:
    temp = pd.DataFrame(columns=['point_id','Depth','bulk_density'])
    temp['point_id'] = bd2018['POINT_ID']
    temp['Depth'] = col.split(' ')[1] + ' cm'
    temp['bulk_density'] = bd2018[col]
    bd = pd.concat([bd,temp],axis=0).reset_index(drop=True)
    
bd = bd.dropna(subset=['bulk_density']).reset_index(drop=True)
bd = bd.merge(lucas2018[['point_id','latg','long']], on=['point_id'], how='left')

result = lucas2018.merge(bd, on=['point_id','latg','long','Depth'], how='outer')

result['hzn_top'] = result['Depth'].str.split('-').str[0]
result['hzn_btm'] = result['Depth'].str.split('-').str[1].str[0:2]
result['time'] = 2018
result = result.drop(columns='Depth')
result.to_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.2018_harmonized.csv',index=False)


### lucas swiss

In [8]:
# lucas swiss
swiss = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/LUCAS2015_topsoildata_20200323/LUCAS_2015_Topsoil_data_of_Switzerland-with-coordinates.csv')
swiss = swiss.iloc[:, :-12] # exclude the other method, keep the spade one, which aligns with LUCAS
swiss.columns = swiss.iloc[0] # get the correct column names
swiss = swiss.loc[1::]
swiss = swiss.rename(columns={'Soil_ID':'point_id','pH in H2O\n':'ph_h2o','pH in CaCl2':'ph_cacl2','Phosphorus/\nmg kg–1':'P', 'Land cover':'lc_survey',
                              'Potassium/\nmg kg–1':'K','Organic carbon/\ng kg–1':'oc','Calcium carbonate/\ng kg–1':'caco3','Nitrogen/\ng kg–1':'N',
                              'GPS_ LAT':'lat','GPS_LONG':'lon','Clay/\ng kg–1':'clay','Silt/\ng kg–1':'silt','Sand/\ng kg–1':'sand','Electrical conductivity/\nmS m–1':'EC'})
swiss['time'] = 2015
swiss['ref'] = 'LUCAS-swiss'
swiss['nuts0'] = 'CH'
swiss['hzn_top'] = 0
swiss['hzn_btm'] = 20
swiss.to_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.swiss_harmonized.csv',index=False)


## merge all the lucas data

In [62]:
# merge all the lucas data (except for swiss)
df = pd.DataFrame(columns=['point_id','long','latg','time','hzn_top','hzn_btm','clay','silt','sand','oc','caco3','N',
                              'P','K','ph_h2o','ph_cacl2','bulk_density','EC','CEC'])
print(len(df.columns))
names = ['2009','2012','2015','2018']
for nn in names:
    temp = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.{nn}_harmonized.csv')
    df = pd.concat([df,temp],axis=0).reset_index(drop=True)
    print(nn,len(df.columns))
    
df['point_id'] = pd.to_numeric(df['point_id'], errors='coerce')
df = df.dropna(subset=['point_id'],how='any')
df['point_id'] = df['point_id'].astype(int)

# get the real coordinates from SOIL MICRO data, GPS_*
df = df.reset_index()
tt = [2009,2012,2015,2018]
df['lat'] = 0
df['lc_survey'] = 0
df['lon'] = 0
df['nuts0'] = 0
for i in tt:
    l = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas_microdata/EU_{i}_20200213.CSV',low_memory=False)
    l = l[['GPS_LAT','GPS_LONG','POINT_ID','LC1','NUTS0','GPS_EW']]
    l.loc[(l['GPS_EW']=='W') | (l['GPS_EW']==2),'GPS_LONG'] = l.loc[(l['GPS_EW']=='W') | (l['GPS_EW']==2),'GPS_LONG']*(-1)
    l['POINT_ID'] = pd.to_numeric(l['POINT_ID'], errors='coerce')
    l = l.rename(columns={'POINT_ID':'point_id'})
    
    df = df.merge(l,how='left',on='point_id')
    
    df.loc[df['time']==i,'lat'] = df.loc[df['time']==i,'GPS_LAT']
    df.loc[df['time']==i,'lon'] = df.loc[df['time']==i,'GPS_LONG']
    df.loc[df['time']==i,'nuts0'] = df.loc[df['time']==i,'NUTS0']
    df.loc[df['time']==i,'lc_survey'] = df.loc[df['time']==i,'LC1']
    df = df.drop(columns=['GPS_LAT','GPS_LONG','LC1','NUTS0','GPS_EW'])


# mend the missing coordinates, and drop those without coordinates information
df.loc[df['lon'].isna(),'lat'] = df.loc[df['lon'].isna(),'latg']
df.loc[df['lon'].isna(),'lon'] = df.loc[df['lon'].isna(),'long']
df.loc[df['lat']>70,'lon'] = df.loc[df['lat']>70,'long']
df.loc[df['lat']>70,'lat'] = df.loc[df['lat']>70,'latg']

# mend data without explicit nuts0 info
df.loc[df['nuts0'].isna(),'nuts0'] = 'NL'

# clean the data according to coordinates, depth, and time
ll = len(df)
df = df.loc[~df['lat'].isna()]
print(f'{ll-len(df)} rows without valid coordinates recorded')
df['ref'] = 'LUCAS'

# add swiss
swiss = pd.read_csv(f'{data_path}/raw_data/LUCAS_TOPSOIL/lucas.swiss_harmonized.csv')
swiss['ref'] = 'LUCAS-swiss'
df = pd.concat([df,swiss])


# keep only necessary columns
rm = ['coarse','Notes','sample_ID','X_LAEA','Y_LAEA','SoilID','Coarse','LC','LU','NUTS_0','NUTS_1','NUTS_2','Sampling location',
      'NUTS_3','LC0_Desc','LC1_Desc','LU1_Desc','geometry','Ox_Al','Ox_Fe','SURVEY_DATE','Elev','latg','long','index']
df = df.drop(columns=rm)


# save the data
df.to_csv(f'{data_path}/data/lucas.full_harmonized.csv',index=False)


19
2009 22
2012 25
2015 36
2018 40
58 rows without valid coordinates recorded


### overview

In [63]:
# overview of soil property availability
for col in df.columns.values.tolist():
    print(f'{col}: missing {df[col].isna().sum()} data, {round(df[col].isna().sum()*100/len(df))}%')


point_id: missing 0 data, 0%
time: missing 0 data, 0%
hzn_top: missing 0 data, 0%
hzn_btm: missing 0 data, 0%
clay: missing 49214 data, 65%
silt: missing 49214 data, 65%
sand: missing 49214 data, 65%
oc: missing 12214 data, 16%
caco3: missing 20348 data, 27%
N: missing 12493 data, 17%
P: missing 12519 data, 17%
K: missing 12493 data, 17%
ph_h2o: missing 12493 data, 17%
ph_cacl2: missing 12493 data, 17%
bulk_density: missing 56967 data, 75%
EC: missing 34337 data, 46%
CEC: missing 53624 data, 71%
lat: missing 0 data, 0%
lc_survey: missing 1 data, 0%
lon: missing 0 data, 0%
nuts0: missing 0 data, 0%
ref: missing 0 data, 0%


In [64]:
df


,point_id,time,hzn_top,hzn_btm,clay,silt,sand,oc,caco3,N,...,ph_h2o,ph_cacl2,bulk_density,EC,CEC,lat,lc_survey,lon,nuts0,ref
0,57981484,2009,0,20,21.0,20.0,60.0,13.7,26.0,1.6,...,8.21,7.39,NaN,NaN,19.2,35.01095,D20,26.14024,EL,LUCAS
1,57941494,2009,0,20,28.0,61.0,10.0,34.0,4.0,3.0,...,7.45,6.70,NaN,NaN,31.2,35.10740,D20,26.11719,EL,LUCAS
2,57841488,2009,0,20,29.0,59.0,12.0,28.1,4.0,2.7,...,7.79,7.12,NaN,NaN,32.5,35.07291,D10,25.99834,EL,LUCAS
3,57821498,2009,0,20,17.0,29.0,54.0,23.7,22.0,2.4,...,7.84,7.18,NaN,NaN,15.4,35.16551,B81,25.99658,EL,LUCAS
4,57621488,2009,0,20,28.0,55.0,16.0,38.4,80.0,3.1,...,7.97,7.32,NaN,NaN,34.5,35.11394,D10,25.76281,EL,LUCAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,53156,2015,0,20,17.0,45.0,39.0,29.0,20,3.3,...,7.12,7.00,NaN,24.9,NaN,47.63880,meadow,8.68220,CH,LUCAS-swiss
146,53157,2015,0,20,31.0,52.0,17.0,33.3,207,3.1,...,7.54,7.30,NaN,29.2,NaN,47.64200,pasture,9.02920,CH,LUCAS-swiss
147,53158,2015,0,20,25.0,36.0,39.0,17.9,36,2.0,...,7.82,7.40,NaN,23.2,NaN,47.67520,cropland,8.75810,CH,LUCAS-swiss
148,53159,2015,0,20,18.0,46.0,36.0,23.6,11,2.9,...,7.45,7.20,NaN,40.8,NaN,47.71180,cropland,8.81340,CH,LUCAS-swiss
